In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

2024-05-26 09:30:54.524384: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-26 09:30:54.552601: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-26 09:30:54.552630: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-26 09:30:54.553665: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-26 09:30:54.559180: I tensorflow/core/platform/cpu_feature_guar

In [2]:
df = pd.read_csv('household_power_consumption.csv')
df

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.84,18.4,0,1,17.0
1,16/12/2006,17:25:00,5.36,0.436,233.63,23,0,1,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.29,23,0,2,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.74,23,0,1,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.68,15.8,0,1,17.0
...,...,...,...,...,...,...,...,...,...
1048570,13/12/2008,21:34:00,0.426,0.076,242.27,1.8,0,0,0.0
1048571,13/12/2008,21:35:00,0.424,0.076,242.1,1.8,0,0,0.0
1048572,13/12/2008,21:36:00,0.422,0.076,241.73,1.8,0,0,0.0
1048573,13/12/2008,21:37:00,0.422,0.078,242.56,1.8,0,0,0.0


In [3]:
df.dropna(inplace=True) # drop missing values
df.insert(0,'Datetime',pd.to_datetime(df['Time'],dayfirst=True)) # merging Date and Time columns
df.drop(columns=['Date','Time'],inplace=True) # Dropping The Date and Time columns
df.set_index('Datetime',inplace=True) # Set Datetime values as index
df

/tmp/ipykernel_14638/3865229193.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.insert(0,'Datetime',pd.to_datetime(df['Time'],dayfirst=True)) # merging Date and Time columns


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Datetime,,,,,,,
2024-05-26 17:24:00,4.216,0.418,234.84,18.4,0,1,17.0
2024-05-26 17:25:00,5.36,0.436,233.63,23,0,1,16.0
2024-05-26 17:26:00,5.374,0.498,233.29,23,0,2,17.0
2024-05-26 17:27:00,5.388,0.502,233.74,23,0,1,17.0
2024-05-26 17:28:00,3.666,0.528,235.68,15.8,0,1,17.0
...,...,...,...,...,...,...,...
2024-05-26 21:34:00,0.426,0.076,242.27,1.8,0,0,0.0
2024-05-26 21:35:00,0.424,0.076,242.1,1.8,0,0,0.0
2024-05-26 21:36:00,0.422,0.076,241.73,1.8,0,0,0.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1044506 entries, 2024-05-26 17:24:00 to 2024-05-26 21:38:00
Data columns (total 7 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Global_active_power    1044506 non-null  object 
 1   Global_reactive_power  1044506 non-null  object 
 2   Voltage                1044506 non-null  object 
 3   Global_intensity       1044506 non-null  object 
 4   Sub_metering_1         1044506 non-null  object 
 5   Sub_metering_2         1044506 non-null  object 
 6   Sub_metering_3         1044506 non-null  float64
dtypes: float64(1), object(6)
memory usage: 63.8+ MB


In [5]:
for i, col in enumerate(df.columns):
    print('The type of '+col+' is ',type(df[col].head(1).values[0]))

The type of Global_active_power is  <class 'str'>
The type of Global_reactive_power is  <class 'str'>
The type of Voltage is  <class 'str'>
The type of Global_intensity is  <class 'str'>
The type of Sub_metering_1 is  <class 'str'>
The type of Sub_metering_2 is  <class 'str'>
The type of Sub_metering_3 is  <class 'numpy.float64'>


In [6]:
for i in df.columns:
    df[i] = (df[i]).astype('float64')
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1044506 entries, 2024-05-26 17:24:00 to 2024-05-26 21:38:00
Data columns (total 7 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Global_active_power    1044506 non-null  float64
 1   Global_reactive_power  1044506 non-null  float64
 2   Voltage                1044506 non-null  float64
 3   Global_intensity       1044506 non-null  float64
 4   Sub_metering_1         1044506 non-null  float64
 5   Sub_metering_2         1044506 non-null  float64
 6   Sub_metering_3         1044506 non-null  float64
dtypes: float64(7)
memory usage: 63.8 MB


In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df)

In [8]:
def create_sequences(data, sequence_length, prediction_length):
    sequences = []
    labels = []
    for i in range(len(data) - sequence_length - prediction_length + 1):
        seq = data[i:i + sequence_length]
        label = data[i + sequence_length:i + sequence_length + prediction_length]
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

sequence_length = 60  # e.g., use the past 60 minutes to predict the next 5 minutes
prediction_length = 5  # predict next 5 minutes
X, y = create_sequences(scaled_data, sequence_length, prediction_length)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Reshape

model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(units=y_train.shape[1] * y_train.shape[2]))
model.add(Reshape((y_train.shape[1], y_train.shape[2])))

model.compile(optimizer='adam', loss='mean_squared_error')

In [11]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
20889/20889 [==============================] - 446s 21ms/step - loss: 0.0024 - val_loss: 0.0022
Epoch 2/10
20889/20889 [==============================] - 448s 21ms/step - loss: 0.0021 - val_loss: 0.0020
Epoch 3/10
20889/20889 [==============================] - 441s 21ms/step - loss: 0.0020 - val_loss: 0.0020
Epoch 4/10
20889/20889 [==============================] - 454s 22ms/step - loss: 0.0019 - val_loss: 0.0020
Epoch 5/10
20889/20889 [==============================] - 499s 24ms/step - loss: 0.0019 - val_loss: 0.0019
Epoch 6/10
20889/20889 [==============================] - 472s 23ms/step - loss: 0.0018 - val_loss: 0.0019
Epoch 7/10
20889/20889 [==============================] - 476s 23ms/step - loss: 0.0018 - val_loss: 0.0018
Epoch 8/10
20889/20889 [==============================] - 464s 22ms/step - loss: 0.0018 - val_loss: 0.0018
Epoch 9/10
20889/20889 [==============================] - 477s 23ms/step - loss: 0.0018 - val_loss: 0.0018
Epoch 10/10
20889/20889 [============

In [12]:
y_pred = model.predict(X_test)


6528/6528 [==============================] - 57s 9ms/step


In [13]:
import numpy as np

# Inverse transform the predictions and the true values to their original scale
y_pred_rescaled = scaler.inverse_transform(y_pred.reshape(-1, y_pred.shape[-1]))
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, y_test.shape[-1]))

# Reshape back to original shape for multi-step predictions
y_pred_rescaled = y_pred_rescaled.reshape(-1, prediction_length, y_test.shape[-1])
y_test_rescaled = y_test_rescaled.reshape(-1, prediction_length, y_test.shape[-1])

# Print the decoded predictions
for i in range(y_pred_rescaled.shape[1]):
    print(f'Sub-Meter {i+1} Next 5 Minute Prediction:')
    print(y_pred_rescaled[:, i])


Sub-Meter 1 Next 5 Minute Prediction:
[[ 1.72188425e+00  1.54981017e-01  2.38100067e+02 ... -2.33203173e-02
   4.05089349e-01  1.73773289e+01]
 [ 2.77171791e-01  4.30511162e-02  2.39426529e+02 ... -3.10491025e-01
  -3.79807502e-03  1.03569284e-01]
 [ 2.99934983e-01  1.81851238e-01  2.38703598e+02 ...  7.94529915e-04
   1.18551230e+00  2.68565893e-01]
 ...
 [ 1.42657709e+00  1.11640558e-01  2.40949524e+02 ... -1.18681192e-01
  -7.20875859e-02  1.75501194e+01]
 [ 3.96884292e-01  1.59872491e-02  2.44814804e+02 ... -3.12997401e-01
  -1.69976279e-02  3.65917057e-01]
 [ 2.23837972e-01  7.20643550e-02  2.38554474e+02 ... -2.45361030e-01
   7.70390868e-01  1.14491299e-01]]
Sub-Meter 2 Next 5 Minute Prediction:
[[ 1.71451485e+00  1.45909578e-01  2.38252792e+02 ... -1.34960413e-01
   5.31090677e-01  1.73997002e+01]
 [ 2.72847623e-01  3.60492691e-02  2.39529587e+02 ... -2.95982361e-01
  -1.38803929e-01  1.09897345e-01]
 [ 2.66656399e-01  1.53550848e-01  2.38819916e+02 ... -9.21612978e-02
   1.055

In [14]:
last_sequence = scaled_data[-sequence_length:]
last_sequence = np.expand_dims(last_sequence, axis=0)
next_five_minute_prediction = model.predict(last_sequence)

# Decode the predictions
next_five_minute_prediction_rescaled = scaler.inverse_transform(next_five_minute_prediction.reshape(-1, next_five_minute_prediction.shape[-1]))

# Reshape the prediction to separate sub-meters
next_five_minute_prediction_rescaled = next_five_minute_prediction_rescaled.reshape(prediction_length, -1)

# Print predictions for each sub-meter
for i in range(next_five_minute_prediction_rescaled.shape[1]):
    print(f'Sub-Meter {i+1} Next 5 Minute Prediction:')
    print(next_five_minute_prediction_rescaled[:, i])


1/1 [==============================] - 0s 22ms/step
Sub-Meter 1 Next 5 Minute Prediction:
[0.4433099  0.4376333  0.4723794  0.48571533 0.49099523]
Sub-Meter 2 Next 5 Minute Prediction:
[0.08307143 0.08497218 0.08641165 0.08595325 0.08667104]
Sub-Meter 3 Next 5 Minute Prediction:
[242.30289 242.2711  242.15468 242.10591 242.06055]
Sub-Meter 4 Next 5 Minute Prediction:
[1.8687122 1.8643268 2.0092902 2.076496  2.0877001]
Sub-Meter 5 Next 5 Minute Prediction:
[-0.31566173 -0.39763272 -0.17871618 -0.07064819 -0.08900046]
Sub-Meter 6 Next 5 Minute Prediction:
[0.14080743 0.0288928  0.04770593 0.06906127 0.03048775]
Sub-Meter 7 Next 5 Minute Prediction:
[0.16674134 0.18614404 0.19231966 0.30354184 0.44225845]
